## 0. Set up

In [1]:
import matplotlib.pyplot as plt
# TODO: Include Pandas in image

ModuleNotFoundError: No module named 'pandas'

In [1]:
data_dir='/home/jovyan/data'

## 1. QC of 1000G VCF

##### **(OPTIONAL)** Print fields from VCF

In [ ]:
%%bash -s "$data_dir"

data_dir=$1

bcftools view -h $data_dir/GWD_30x_calls.vcf.gz | grep "^##.*INFO\|^##FORMAT"

# NOTE
# QUAL - Phred-scaled quality score for the assertion made in ALT. 
# See https://samtools.github.io/hts-specs/VCFv4.1.pdf

#### 1.1 Extract fields from VCF, save as table

In [2]:
%%bash -s "$data_dir"

data_dir=$1

bcftools query -f '%POS\t%DP\t%QUAL\t%VDB\t%SGB\t%RPB\t%MQB\t%MQSB\t%BQB\t%MQ0F\t%AC\t%AN\t%MQ\n' $data_dir/GWD_30x_calls.vcf.gz > $data_dir/GWD_30x_calls.variant_info.tsv

#### 1.2 Plot fields

In [ ]:
info = pd.read_csv(
    f'{data_dir}/GWD_30x_calls.variant_info.tsv', 
    sep='\t', 
    names=['pos','dp','qual','vdb','sgb','rpb','mqb','mqsb','bqb','mq0f','ac', 'an','mq'],
    na_values=['.']
)

In [ ]:
field_title_dict = {
    'dp': 'Raw read depth',
    'qual': 'Phred-scaled quality score for the assertion made in ALT',
    'vdb': 'Variant Distance Bias for filtering splice-site artefacts in RNA-seq data\n(bigger is better)',
    'rpb': 'Mann-Whitney U test of Read Position Bias\n(bigger is better)',
    'mqb': 'Mann-Whitney U test of Mapping Quality Bias\n(bigger is better)',
    'bqb': 'Mann-Whitney U test of Base Quality Bias\n(bigger is better)',
    'mqsb': 'Mann-Whitney U test of Mapping Quality vs Strand Bias\n(bigger is better)',
    'sgb': 'Segregation based metric',
    'mq0f': 'Fraction of MQ0 reads\n(smaller is better)',
    'ac': 'Allele count in genotypes for each ALT allele, in the same order as listed',
    'an': 'Total number of alleles in called genotypes',
    'mq': 'Average mapping quality'
}

In [ ]:
fig, axs = plt.subplots(figsize=(24,16), nrows=3, ncols=4)

for i, f in enumerate(field_title_dict.keys()):
    _ = axs[i//4,i%4].hist(info[f], 50)
    _ = axs[i//4,i%4].set_xlabel(f.upper())
    axs[i//4,i%4].set_title(f.upper())

plt.tight_layout()

plt.suptitle('Variant info: GWD_30x_calls.vcf.gz', size=15)
plt.savefig('variant_info.png',dpi=300)

#### 1.3 Set cutoffs

In [64]:
# include_variants_dict = {
#     'dp': ('>', 1000), #'Raw read depth'
#     'qual': ('>', 50), #'Phred-scaled quality score for the assertion made in ALT',
#     'vdb': ('>', 1e-4), #'Variant Distance Bias for filtering splice-site artefacts in RNA-seq data\n(bigger is better)',
#     'rpb': ('>', 1e-5), # 'Mann-Whitney U test of Read Position Bias\n(bigger is better)',
#     'mqb': ('>', 1e-5), #'Mann-Whitney U test of Mapping Quality Bias\n(bigger is better)',
#     'bqb': ('>', 1e-5), #'Mann-Whitney U test of Base Quality Bias\n(bigger is better)',
#     'mqsb': ('>', 1e-8), #'Mann-Whitney U test of Mapping Quality vs Strand Bias\n(bigger is better)',
#     # NOT SURE WHICH WAY TO FILTER ON SGB
#     'sgb': ('', None), #'Segregation based metric',
#     'mq0f': ('<', 0.1), #'Fraction of MQ0 reads\n(smaller is better)',
#     'ac': ('', None), #'Allele count in genotypes for each ALT allele, in the same order as listed',
#     'an': ('>', 300), #'Total number of alleles in called genotypes',
#     'mq': ('>', 50)#'Average mapping quality'
# }

include_variants_dict = {
    'dp': ('>', 1000), #'Raw read depth'
    'qual': ('>', 50), #'Phred-scaled quality score for the assertion made in ALT',
    'vdb': ('>', 1e-6), #'Variant Distance Bias for filtering splice-site artefacts in RNA-seq data\n(bigger is better)',
    'rpb': ('>', 1e-6), # 'Mann-Whitney U test of Read Position Bias\n(bigger is better)',
    'mqb': ('>', 1e-6), #'Mann-Whitney U test of Mapping Quality Bias\n(bigger is better)',
    'bqb': ('>', 1e-6), #'Mann-Whitney U test of Base Quality Bias\n(bigger is better)',
    'mqsb': ('>', 1e-9), #'Mann-Whitney U test of Mapping Quality vs Strand Bias\n(bigger is better)',
    # NOT SURE WHICH WAY TO FILTER ON SGB
    'sgb': ('', None), #'Segregation based metric',
    'mq0f': ('<', 0.1), #'Fraction of MQ0 reads\n(smaller is better)',
    'ac': ('', None), #'Allele count in genotypes for each ALT allele, in the same order as listed',
    'an': ('>', 300), #'Total number of alleles in called genotypes',
    'mq': ('>', 50)#'Average mapping quality'
}

# String containing all the filter logic to be used by bcftools
logic = ' & '.join([f'{field.upper()}{equality}{threshold}' for field, (equality, threshold) in include_variants_dict.items() if threshold is not None])


#### 1.4 Filter variants

##### **(Optional)** Filter variants in the Pandas DataFrame

In [ ]:
for field, (equality, threshold) in include_variants_dict.items():
    if threshold is not None:
        if equality == '>':
            info[f'fail_{field}'] = info[field]<=threshold
        elif equality == '<':
            info[f'fail_{field}'] = info[field]>=threshold
            
fail_columns = [c for c in info.columns if 'fail_' in c]

Check if secretor status variant (chr19:48703417) passes QC in the sequencing data

In [ ]:
secretor_position=48703417
assert any(info.loc[info.pos==secretor_position, fail_columns]) is False    


##### 1.4.1 Filter VCF using set cutoffs

Set path to QCed VCF

In [57]:
qced_vcf = f'{data_dir}/GWD_30x_calls.qced.vcf.gz'

Use bcftools to filter and save to a new VCF

In [89]:
%%bash -s "$data_dir" "$logic" "$qced_vcf"

data_dir=$1
logic=$2
qced_vcf=$3

seq_vcf="${data_dir}/GWD_30x_calls.vcf.gz"

original_ct=$( bcftools view -H -G ${seq_vcf} | wc -l )

bcftools filter ${seq_vcf} \
  -Oz \
  -i "${logic}" \
  -o ${qced_vcf} \
&& filtered_ct=$( bcftools view -G -H ${qced_vcf} | wc -l ) \
|| ( echo "Failed." && exit 1 )

if [ $? == 0 ]; then
    echo ">> Variant counts <<"
    echo -e "original $original_ct"
    echo -e "post-qc  $filtered_ct"
    echo -e "removed  $(( original_ct - filtered_ct ))"
fi

>> Variant counts <<
original 15683
post-qc  10051
removed  5632


## 2. Phase 1000G VCF

If needed, download genetic map from
https://storage.googleapis.com/broad-alkesgroup-public/Eagle/downloads/tables/genetic_map_hg38_withX.txt.gz

In [80]:
%%bash -s "$data_dir"

data_dir=$1

if [ ! -f "${data_dir}/genetic_map_hg38_withX.txt.gz" ]; then
    url="https://storage.googleapis.com/broad-alkesgroup-public/Eagle/downloads/tables/genetic_map_hg38_withX.txt.gz"
    wget ${url} \
      --directory-prefix ${data_dir} \
      --no-verbose
fi

Split multiallelic sites before running Eagle

In [90]:
split_vcf=f'{data_dir}/GWD_30x_calls.split.vcf.gz'

In [94]:
%%bash -s "$qced_vcf" "$split_vcf"

qced_vcf=$1
split_vcf=$2

bcftools norm -m - ${qced_vcf} \
  -Oz \
  -o ${split_vcf}

Lines   total/split/realigned/skipped:	10051/42/0/0


In [98]:
%%bash -s "$data_dir" "$split_vcf"

data_dir=$1
split_vcf=$2

geneticMapFile="${data_dir}/genetic_map_hg38_withX.txt.gz"
out="${data_dir}/GWD_30x_calls.qced.phased"

eagle \
  --geneticMapFile ${geneticMapFile} \
  --vcf ${split_vcf} \
  --numThreads 5 \
  --outPrefix ${out} 
  

                      +-----------------------------+
                      |                             |
                      |   Eagle v2.3.5              |
                      |   August 2, 2017            |
                      |   Po-Ru Loh                 |
                      |                             |
                      +-----------------------------+

Copyright (C) 2015-2016 Harvard University.
Distributed under the GNU GPLv3+ open source license.

Command line options:

eagle \
    --geneticMapFile /home/jovyan/data/genetic_map_hg38_withX.txt.gz \
    --vcf /home/jovyan/data/GWD_30x_calls.split.vcf.gz \
    --numThreads 5 \
    --outPrefix /home/jovyan/data/GWD_30x_calls.qced.phased 

Setting number of threads to 5

=== Reading genotype data ===

Reading genotypes for N = 164 samples
Read M = 10093 variants
Filling in genetic map coordinates using reference file:
  /home/jovyan/data/genetic_map_hg38_withX.txt.gz
Physical distance range: 1109949 base pairs
Gene

Check genotypes (first 20 variants in first 3 samples) to see how phasing changes VCFs

In [110]:
!bcftools view -H /home/jovyan/data/GWD_30x_calls.qced.vcf.gz | head -20 | cut -f 10-12

1/1:255,111,0	1/1:255,108,0	1/1:255,105,0
0/0:0,99,255	0/0:0,102,255	0/0:0,111,255
0/0:0,108,255	0/0:0,117,255	0/0:0,102,255
0/0:0,102,255	0/0:0,99,255	0/0:0,96,255
0/0:0,114,255	0/0:0,87,255	0/0:0,111,255
0/0:0,120,255	0/0:0,99,255	0/0:0,93,255
0/0:0,120,255	0/1:181,0,250	0/0:0,117,255
0/0:0,75,255	0/0:0,123,255	0/0:0,120,255
0/0:0,93,255	0/0:0,108,255	0/0:0,141,255
0/0:0,84,255	0/0:0,93,255	0/0:0,102,255
0/0:0,114,255	0/1:219,0,239	0/0:0,111,255
0/1:254,0,203	0/0:0,154,255	0/0:0,111,255
0/0:0,108,255	0/0:0,151,255	0/0:0,111,255
1/1:255,81,0	0/1:243,0,183	1/1:255,102,0
1/1:255,99,0	0/0:0,138,255	0/1:255,0,221
0/0:0,105,255	0/0:0,141,255	0/0:0,141,255
0/0:0,96,255	0/0:0,111,255	0/0:0,105,255
0/0:0,87,255	0/0:0,111,255	0/0:0,111,255
0/0:0,84,255	0/1:255,0,187	0/0:0,114,255
1/1:255,99,0	0/1:158,0,255	1/1:255,126,0
[main_vcfview] Error: cannot write to (null)


In [109]:
!bcftools view -H /home/jovyan/data/GWD_30x_calls.qced.phased.vcf.gz | head -20 | cut -f 10-12


1|1:255,111,0	1|1:255,108,0	1|1:255,105,0
0|0:0,99,255	0|0:0,102,255	0|0:0,111,255
0|0:0,108,255	0|0:0,117,255	0|0:0,102,255
0|0:0,102,255	0|0:0,99,255	0|0:0,96,255
0|0:0,114,255	0|0:0,87,255	0|0:0,111,255
0|0:0,120,255	0|0:0,99,255	0|0:0,93,255
0|0:0,120,255	1|0:181,0,250	0|0:0,117,255
0|0:0,75,255	0|0:0,123,255	0|0:0,120,255
0|0:0,93,255	0|0:0,108,255	0|0:0,141,255
0|0:0,84,255	0|0:0,93,255	0|0:0,102,255
0|0:0,114,255	1|0:219,0,239	0|0:0,111,255
1|0:254,0,203	0|0:0,154,255	0|0:0,111,255
0|0:0,108,255	0|0:0,151,255	0|0:0,111,255
1|1:255,81,0	0|1:243,0,183	1|1:255,102,0
1|1:255,99,0	0|0:0,138,255	1|0:255,0,221
0|0:0,105,255	0|0:0,141,255	0|0:0,141,255
0|0:0,96,255	0|0:0,111,255	0|0:0,105,255
0|0:0,87,255	0|0:0,111,255	0|0:0,111,255
0|0:0,84,255	1|0:255,0,187	0|0:0,114,255
1|1:255,99,0	0|1:158,0,255	1|1:255,126,0
[main_vcfview] Error: cannot write to (null)


## 3. QC GGVP VCF

## 4. Phase GGVP VCF

## 5. Impute GGVP VCF